In [ ]:
%%bash
#pip install TextBlob
#conda install gensim

In [35]:
import pandas as pd
import numpy as np
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
from gensim import corpora, models, similarities
import slugify as sl

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"
movieplots = pd.read_csv(dropbox + 'movieplots.csv')
#this is a work around because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

In [3]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [4]:
movieplots = movieplots.apply(clean, axis = 1)
dftouse = movieplots[movieplots.year > 2014]

In [5]:
print movieplots.shape
print dftouse.shape

(259028, 5)
(16943, 5)


In [6]:
import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)   

In [52]:
plots_sdf = sqlsc.createDataFrame(dftouse)

In [45]:
def slugify (text):
    return sl.slugify(text)

In [8]:
def sentences(text):
    blob = tb.TextBlob(text)
    return blob.sentences

In [9]:
def stem(word):
    return PorterStemmer().stem(word)

In [53]:
num = re.compile('\d')
def contains_number(word):
    return bool(num.search(word))

In [85]:
def process(sents, dostop=True, dostem=True, donum=True, doslug=True):
    stopwords = set(sw.words('english'))
    sents_processed  = []
    words_processed  = []
    for sent in sents:
        words = []
        for word in sent.words:
            if donum and contains_number(word):
                    continue
            if doslug:
                word = slugify(word)
            if dostop and word in stopwords:
                    continue
            if dostem:
                word = stem(word)
            if len(word) < 2:
            #    continue
            words.append(word.lower())
        sents_processed.append(words)
        words_processed += words 
    return (sents_processed, words_processed)

In [55]:
plots = (plots_sdf[['title','plots']]
    .map(lambda x : (' '.join(x[1])))
)
docs = (plots
    .map(sentences)
    .map(process)          
).cache()
docs_sents = docs.map(lambda x : x[0]).collect()
docs_words = docs.map(lambda x : x[1]).collect()

In [58]:
dictionary = corpora.Dictionary(docs_words)
dictionary.save(dropbox +"testdict.dict") 
dictionary.token2id

{u'fawn': 9708,
 u'': 17780,
 u'fawl': 34654,
 u'isis-isil': 14323,
 u'fawk': 19662,
 u'woodi': 9808,
 u'chthonic': 10683,
 u'redzepi': 31383,
 u'sonja': 4484,
 u'showa': 31229,
 u'suzann': 12462,
 u'gavan': 32671,
 u'francesco': 12294,
 u'francesca': 317,
 u'hyrum': 40167,
 u'isaia': 21607,
 u'gad': 40830,
 u'scola': 27693,
 u'canez': 21098,
 u'scold': 6962,
 u'self-referenti': 6164,
 u'xandra': 33383,
 u'vano': 27548,
 u'antigian': 21470,
 u'hermann': 19020,
 u'hermano': 33460,
 u'lencr': 28444,
 u'kleiman': 16911,
 u'raouf': 34862,
 u'pigment': 15797,
 u'dongen': 22509,
 u'stopforth': 40621,
 u'fractal': 34143,
 u'showroom': 35777,
 u'elvi': 13612,
 u'airfar': 26112,
 u'wooden': 16703,
 u'showcas': 4104,
 u'wednesday': 6218,
 u'semi-sup': 35319,
 u'alzeihm': 37989,
 u'crotch': 21759,
 u'jenkin': 22756,
 u'south-west': 34435,
 u're-posit': 33885,
 u'tumer': 39804,
 u'coal-burn': 33243,
 u'thrace': 25922,
 u'one-hundred-year-old': 28503,
 u'frederick': 4254,
 u'super-club': 23564,
 u'

In [57]:
corpus = (dictionary.doc2bow(doc) for doc in docs_words)
corpora.MmCorpus.serialize(dropbox +"testcorpus.mm", corpus)

In [ ]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [70]:
plot=u'a the s test'
sentences(plot)

[Sentence("a the s test")]

In [74]:
plots_sdf.

[Row(title=u'#1 at the Apocalypse Box Office (2015)', source=u'imdb', year=2015.0, plots=[u'Jules is, self declared, the most useless person in the post apocalyptic world, until he finds an old film camera and determines to make the greatest movie in the new world... the only movie in the new world. But his first day filming is proving to be much more difficult than he imagined. Who knew that making a movie after the end of the world would be so hard?'], type=u'user plot'),
 Row(title=u'#50Fathers (2015)', source=u'imdb', year=2015.0, plots=[u'#50Fathers is an American Dramatic Comedy. Dominique Hathaway is the best high school athlete in the nation. "Nique"\xc2\x9d is a three sport All-American and a Blue Chip MLB or NFL draft prospect who will certainly enjoy a lifestyle few young adults are privy too. With that in mind, Nique decides to give his brother and him something they haven\'t had since the death of his parents, but now more than ever is needed... A Father. He post on his Yo

In [77]:
t = u'"Nique"\xc2\x9d is a three sport All-American.'


In [86]:
process(sentences(t))

([[u'niqu', u'three', u'sport', u'all-american']],
 [u'niqu', u'three', u'sport', u'all-american'])